In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import os

In [8]:
data_path = '../data/'
sen_names = pd.read_csv(os.path.join(data_path, "raw/handles.csv"))
tweets = pd.read_csv(os.path.join(data_path, "tweets.csv"))
edges = pd.read_csv(os.path.join(data_path, "edges.csv"))

In [9]:
G = nx.DiGraph()

In [10]:
party_dict = dict(zip(sen_names["Name"], sen_names["Party"]))
for i in sen_names["Name"]:
    G.add_node(i, party = party_dict[i])
G.add_edges_from(list(zip(edges["followed"], edges["following"])))

In [5]:
print(nx.info(G))

Name: 
Type: DiGraph
Number of nodes: 100
Number of edges: 3821
Average in degree:  38.2100
Average out degree:  38.2100


In [ ]:
options = {
    'node_size': 30,
    'width': 0.2,
}
nx.draw_kamada_kawai(G, node_color = ['red' if i[1]['party'] == 'R' else 'blue' for i in G.nodes.data()], **options)

In [7]:
tweets

,Senator Name,Tweets,Date
0,AK Lisa Murkowski,"To promote clean energy, we increased funding ...",2020-01-03
1,AK Lisa Murkowski,We focus on energy activities in the Arctic re...,2020-01-03
2,AK Lisa Murkowski,The President signed into law a set of year-en...,2020-01-03
3,AK Lisa Murkowski,I'm proud of the Senate passage of my bipartis...,2020-01-03
4,AK Lisa Murkowski,The New Year gives us an opportunity to reflec...,2020-01-02
5,AK Lisa Murkowski,Wishing Alaskans a safe and Happy New Year’s E...,2020-01-01
6,AK Lisa Murkowski,I’m proud that my Interior-Environment appropr...,2019-12-30
7,AK Lisa Murkowski,"In Alaska, communities like Gustavus, Fairbank...",2019-12-30
8,AK Lisa Murkowski,I was honored to receive the @CFSIUpdate 2019 ...,2019-12-29
9,AK Lisa Murkowski,"During my years in public service, I have alwa...",2019-12-29
